In [1]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *
from ipynb.fs.full.CPv1 import *
import numpy as np
from copy import copy 
from collections import deque
import glob

In [2]:
dr_count = 0

In [3]:
class lock_map: # Class to keep track of all the variables it operated upon
    def __init__(self,init_vc):
        self.lock_vc = init_vc
        self.read_objects = {}
        self.write_objects = {}
    
    def __str__(self): # Converts data of object to string to print
        write_str = ""
        for wr_obj in self.write_objects.keys():
            write_str += (wr_obj+":"+ str(self.write_objects[wr_obj])+", ")
        read_str = ""
        for rd_obj in self.read_objects.keys():
            read_str += (rd_obj+" : " + str(self.read_objects[rd_obj])+", ")
        return "Main Clock : %s\nObject Read Clocks : {%s}\nWrite Clocks : {%s}\n" % (self.lock_vc,read_str,write_str)
    
class object_vc: # To store the VC for each variable along with the thread Id
    def __init__(self,l_vc,tid):
        self.vc = l_vc
        self.tid = tid
    
    def __str__(self):
        return "Clock : %s\t Thread: %s" % (self.vc,self.tid)

In [4]:
def vc_cp_ordered(vc1,vc2): # Checks if vc1 CPs vc2. Returns true if vc2 does not CP vc1
    r = False
    ct = 0
    for i in range(len(vc1)):
        if vc2[i] > vc1[i]:
            return True
        elif vc2[i] == vc1[i]:
            ct+=1
    return ct==len(vc1)

def check_rule1(evt,evt_idx): # Checks if the current is in CP with previous CS if any. Returns True if there is a CP relation
    cs_idx_list = 0
    for cs_idx in lock_access_list[evt.var_name]:
        if evt_idx in critical_section_list[cs_idx].event_idx:
            current_cs_idx = cs_idx
            break
        cs_idx_list+=1
    if cs_idx_list>0:# Check if in CP or from same thread then CP
        if is_cp(cs_idx,lock_access_list[evt.var_name][cs_idx_list-1],critical_section_list) or critical_section_list[cs_idx].tid == critical_section_list[lock_access_list[evt.var_name][cs_idx_list-1]].tid:
            return True

def find_critical_section_index(evt_idx): # Returns CS index for the event for the lock suign the help of lock list
    for cs_idx in lock_access_list[events[evt_idx].var_name]:
        if evt_idx in critical_section_list[cs_idx].event_idx:
            return cs_idx

def merge_maps(idx,map_to,map_from):# Copies values that were used in the child critical section to parent critical section
    cs_idx = find_critical_section_index(idx)
    for k in map_from.read_objects.keys():
        if k in critical_section_list[cs_idx].read_objects:
            map_to.read_objects[k] = map_from.read_objects[k]
    for k in map_from.write_objects.keys():
        if k in critical_section_list[cs_idx].write_objects:
            map_to.write_objects[k] = map_from.write_objects[k]
    map_to.lock_vc = merge_vc(map_to.lock_vc,map_from.lock_vc.copy())
    #map_to.lock_vc = map_from.lock_vc.copy() # can use merge 
    
def get_vc_var(v_name,t_vc,l_map,t_stk): # Returns a vector clock for given variable.
    if len(t_stk) == 0: 
        return t_vc.copy() # If no lock acquired then thread's VC
    else:
        return l_map.lock_vc.copy() # If lock acquired then lock's VC
    
def add_current_vc(evt, t_map, t_vc, l_stk,s_l_map): # Adds current VC to the super lock map
    if len(l_stk) == 0:
        s_l_map[evt.var_name] = object_vc(t_vc,evt.tid) #Under no lock the variable is used as key in map
    else: # If lock is acquired then update it under the lock it was acquired
        if evt.e_type == events_list.Read.value:
            s_l_map[l_stk[-1]].read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
        elif evt.e_type == events_list.Write.value:
            s_l_map[l_stk[-1]].write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            
def compare_vc_to_other_locks(evt,compare_map,v_vc,t_stk): # Compare the VC of the current access to previous access from lock map. Excludes checking for same lock and same thread VC updates
    ret = False
    for k in compare_map.keys():
        if k == evt.var_name: # Check if currently accessed under any lock and earlier accessed whithout any lock
#             print("Empty",v_vc,compare_map[evt.var_name])
            if evt.tid != compare_map[evt.var_name].tid:
                ret = ret or vc_cp_ordered(v_vc,compare_map[evt.var_name].vc)
        elif isinstance(compare_map[k],lock_map) and k not in t_stk: # Check if the key is of a lock object then checks if the access was conflicting
            if (evt.e_type == events_list.Read.value and evt.var_name in compare_map[k].write_objects.keys()) or evt.var_name in compare_map[k].write_objects.keys():
#                 print("Lock Write",v_vc,compare_map[k].write_objects[evt.var_name].vc,t_stk,k)
                if evt.tid != compare_map[k].write_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].write_objects[evt.var_name].vc)
            elif evt.e_type == events_list.Write.value and evt.var_name in compare_map[k].read_objects.keys():
#                 print("Lock Read",v_vc,compare_map[k].write_objects[evt.var_name].vc,t_stk,k)
                if evt.tid != compare_map[k].read_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].read_objects[evt.var_name].vc)
    return ret

def merge_vc(vc1,vc2): # Merges two VC by taking maximum from the both VC and updating in vc1
    for i in range(0,len(vc1)):
        vc1[i] = max(vc1[i],vc2[i])
    return vc1

def show(print_locks_map): # Helper method to print the status of lock map
            print("Locks Map : ")
            for k in print_locks_map.keys():
                print(k," : ",print_locks_map[k])
            print("\n")

In [5]:
def process_all_events_queue(events,event_queue,lock_name,local_lock_stack,local_map_stack,super_locks_map,thread_vc): # Processes all events in the queue of the thread till release of the lock
    global dr_count
    while(len(event_queue) > 0):
        temp_map = None
#         print("Event Queue : ",event_queue,local_lock_stack,thread_vc)
        e_idx = event_queue.popleft()
        if events[e_idx].e_type == events_list.Acquire.value: # Event is lock acquire
            local_lock_stack.append(events[e_idx].var_name)
            if events[e_idx].var_name not in super_locks_map.keys(): # lcok name not present in the super lock map
                super_locks_map[events[e_idx].var_name] = lock_map(thread_vc[events[e_idx].tid].copy())
            else: # Lock name present in super lock map
                super_locks_map[events[e_idx].var_name].lock_vc[events[e_idx].tid] = thread_vc[events[e_idx].tid][events[e_idx].tid]
            if len(local_lock_stack) > 1 and vc_cp_ordered(super_locks_map[events[e_idx].var_name].lock_vc,super_locks_map[local_lock_stack[-2]].lock_vc): # Starting of a child critical section
                super_locks_map[events[e_idx].var_name].lock_vc = merge_vc(super_locks_map[events[e_idx].var_name].lock_vc,super_locks_map[local_lock_stack[-2]].lock_vc.copy())
            if check_rule1(events[e_idx],e_idx): # Check if any CP relation exists from previous CS
                super_locks_map[events[e_idx].var_name].lock_vc[events[e_idx].tid] +=1
        elif events[e_idx].e_type == events_list.Release.value: # EVent is lock release
            if len(local_lock_stack) > 0:
                popped_lock = local_lock_stack.pop() # Pop the lock from top of stack
                if len(local_lock_stack) > 0: # Child CS finished
                    merge_maps(e_idx,super_locks_map[local_lock_stack[-1]],super_locks_map[popped_lock])
                else: # All locks of critical section released
                    if not vc_cp_ordered(super_locks_map[popped_lock].lock_vc,thread_vc[events[e_idx].tid]): # Check if need to update the thread VC as thread VC can be ahead
                        thread_vc[events[e_idx].tid] = merge_vc(thread_vc[events[e_idx].tid],super_locks_map[popped_lock].lock_vc.copy())
                        #thread_vc[events[e_idx].tid] = super_locks_map[popped_lock].lock_vc.copy() # can use merge
            if events[e_idx].var_name == lock_name:
                break
        elif events[e_idx].e_type == events_list.Read.value or events[e_idx].e_type == events_list.Write.value: # Event is either read or write
            if len(local_lock_stack) == 0: # No locks acquired
                var_vc = get_vc_var(events[e_idx].var_name,thread_vc[events[e_idx].tid],super_locks_map,local_lock_stack) # Get VC for the variable
                if compare_vc_to_other_locks(events[e_idx],super_locks_map,var_vc,local_lock_stack):
                    dr_count +=1
                    if dr_count >= 100:
                        return
                    print("DataRace at: \nEvent Index :",e_idx,"\tThread :",events[e_idx].tid,"\tEvent Type :",events[e_idx].e_type," \tVariable : ",events[e_idx].var_name)
                add_current_vc(events[e_idx],None,var_vc,local_lock_stack,super_locks_map)
            else: # Locks acquired
                temp_map = super_locks_map[local_lock_stack[-1]] # Copy values from super lock map
                var_vc = get_vc_var(events[e_idx].var_name,thread_vc[events[e_idx].tid],temp_map,local_lock_stack)
                if compare_vc_to_other_locks(events[e_idx],super_locks_map,var_vc,local_lock_stack):
                    dr_count +=1
                    if dr_count >= 100:
                        return
                    print("DataRace at: \nEvent Index :",e_idx,"\tThread :",events[e_idx].tid,"\tEvent Type :",events[e_idx].e_type," \tVariable : ",events[e_idx].var_name)
                add_current_vc(events[e_idx],temp_map,var_vc,local_lock_stack,super_locks_map)
                super_locks_map[local_lock_stack[-1]] = temp_map # Update super lock map after operating
#         show(super_locks_map)
#     print(local_lock_stack)
                
def check_threads_acquired_lock(events,thread_event_queue,evt_local,local_lock_stack,local_map_stack,super_locks_map,thread_vc): # Finds if there are any other threads that had acquired same lock
    for th_num in range(0,len(thread_event_queue)):
        for th_evt_idx in thread_event_queue[th_num]:
            if (events[th_evt_idx].e_type == events_list.Acquire.value or events[th_evt_idx].e_type == events_list.Release.value) and events[th_evt_idx].var_name == evt_local.var_name:
                process_all_events_queue(events,thread_event_queue[th_num],evt_local.var_name,local_lock_stack[events[th_evt_idx].tid],local_map_stack,super_locks_map,thread_vc)
                break
                
def check_conflicting(evt1,evt2): # Checks if two events are conflicting
    if (evt1.e_type == events_list.Write.value or evt2.e_type == events_list.Write.value) and evt1.var_name == evt2.var_name and evt1.tid != evt2.tid: # Any one of them is write
        return True
    else:
        return False
    
def check_threads_acccess_variable(events,thread_event_queue,evt_local): # Check if two threads access same variable
    for th_num in range(0,len(thread_event_queue)):
        global dr_count
        for th_evt_idx in thread_event_queue[th_num]:
            if check_conflicting(events[th_evt_idx],evt_local):
                dr_count +=1
                if dr_count >= 100:
                    return
                print("DataRace at: \nEvent Index :",th_evt_idx,"\tThread :",evt_local.tid,"\tEvent Type :",evt_local.e_type," \tVariable : ",evt_local.var_name)
                

In [6]:
def CP_V2(events,critical_section_list,num_threads,lock_access_list): # Version 2 implementation starts here
    global dr_count
    dr_count = 0
    thread_lock_stack = []
    thread_map_stack = []
    thread_event_queue = deque([])
    thread_vc = np.zeros((num_threads,num_threads))
    for i in range(num_threads):
        thread_lock_stack.append([])
        thread_map_stack.append([])
        thread_event_queue.append(deque([]))
    evt_idx = 0
    super_locks_map = {}
    
    for e in events:
#         print("Event Number : ",evt_idx)
        temp_map = None
        if e.e_type == events_list.Read.value or e.e_type == events_list.Write.value:
            check_threads_acccess_variable(events,thread_event_queue,e) # Checks if any other thread has same varibale in its stack
            thread_event_queue[e.tid].append(evt_idx)
        elif e.e_type == events_list.Acquire.value:
            check_threads_acquired_lock(events,thread_event_queue,e,thread_lock_stack,thread_map_stack,super_locks_map,thread_vc) # Checks if any other thread has same lock in stack
            thread_event_queue[e.tid].append(evt_idx)
        elif e.e_type == events_list.Release.value:
            thread_event_queue[e.tid].append(evt_idx)
        evt_idx+=1
        if dr_count >= 100:
            return
#         print(thread_event_queue,"\n",thread_lock_stack)
#     print("Finished Reading events. Working on leftovers.===================================================")
    for i in range(num_threads):
        process_all_events_queue(events,thread_event_queue[i],None,thread_lock_stack[i],thread_map_stack[i],super_locks_map,thread_vc)

In [7]:
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("TestCases/CP_Paper_TC/Figure1.log")
events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("TestCases/Figure4b.log")
# cp_matrix_v2 = pass2(events,critical_section_list,lock_access_list)
for x in critical_section_list:
    print(x)

print(num_threads,lock_access_list)
CP_V2(events,critical_section_list,num_threads,lock_access_list)

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'test/Figure4b_x'} 
Events : {0} 
Child Sections : set()

Thread : 1 
Locks : {'0', '2', '1'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_oVar', 'test/Figure4b_pVar'} 
Events : {1, 2, 3, 4, 5, 6, 7, 8} 
Child Sections : {2, 3}

Thread : 1 
Locks : {'1'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_oVar'} 
Events : {2, 3, 4} 
Child Sections : set()

Thread : 1 
Locks : {'2'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_pVar'} 
Events : {5, 6, 7} 
Child Sections : set()

Thread : 2 
Locks : {'3', '4', '1'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_qVar', 'test/Figure4b_oVar'} 
Events : {16, 9, 10, 11, 12, 13, 14, 15} 
Child Sections : {5, 6}

Thread : 2 
Locks : {'4'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_qVar'} 
Events : {10, 11, 12} 
Child Sections : set()

Thread : 2 
Locks : {'1'} 
Read Objects : set() 
Write Objects : {'test/Figure4b_oVar'} 
Events : {13, 14

In [8]:
for filename in glob.glob("TestCases/CP_Paper_TC/*.log"):
    print(filename,"--------------------------------------------------------------------------")
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(events,critical_section_list,num_threads,lock_access_list)

TestCases/CP_Paper_TC/Figure3a.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 8 	Thread : 2 	Event Type : 0  	Variable :  count
TestCases/CP_Paper_TC/Figure1.log --------------------------------------------------------------------------
TestCases/CP_Paper_TC/Figure9.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 8 	Thread : 2 	Event Type : 1  	Variable :  x
TestCases/CP_Paper_TC/Figure3b.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 4 	Thread : 2 	Event Type : 0  	Variable :  count
DataRace at: 
Event Index : 3 	Thread : 2 	Event Type : 1  	Variable :  count
DataRace at: 
Event Index : 4 	Thread : 2 	Event Type : 1  	Variable :  count
DataRace at: 
Event Index : 5 	Thread : 2 	Event Type : 0  	Variable :  count
TestCases/CP_Paper_TC/Figure6.log ----------------------------------------------------------------

In [9]:
for filename in glob.glob("TestCases/TestCase*.log"):
    print(filename,"--------------------------------------------------------------------------")
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(events,critical_section_list,num_threads,lock_access_list)

TestCases/TestCase2.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 0 	Thread : 2 	Event Type : 1  	Variable :  phi_x
DataRace at: 
Event Index : 1 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase6.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 1 	Thread : 2 	Event Type : 1  	Variable :  phi_x
DataRace at: 
Event Index : 4 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase4.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 1 	Thread : 2 	Event Type : 1  	Variable :  phi_x
DataRace at: 
Event Index : 2 	Thread : 2 	Event Type : 1  	Variable :  phi_x
TestCases/TestCase5.log --------------------------------------------------------------------------
TestCases/TestCase7.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 5 	Thread

In [10]:
for filename in glob.glob("TestCases/Figure*.log"):
    print(filename,"--------------------------------------------------------------------------")
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(events,critical_section_list,num_threads,lock_access_list)

TestCases/Figure1.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 7 	Thread : 2 	Event Type : 0  	Variable :  test/Figure1_x
TestCases/Figure3.log --------------------------------------------------------------------------
TestCases/Figure2_2.log --------------------------------------------------------------------------
TestCases/Figure4b.log --------------------------------------------------------------------------
TestCases/Figure9a.log --------------------------------------------------------------------------
TestCases/Figure2_1.log --------------------------------------------------------------------------
TestCases/Figure9c.log --------------------------------------------------------------------------
TestCases/Figure8.log --------------------------------------------------------------------------
TestCases/Figure9b.log --------------------------------------------------------------------------
TestCases/Figure4a.log ---------

In [11]:
filename = "TestCases/Test.log"
print(filename,"--------------------------------------------------------------------------")
events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
CP_V2(events,critical_section_list,num_threads,lock_access_list)

TestCases/Test.log --------------------------------------------------------------------------
DataRace at: 
Event Index : 1 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 3 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 5 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 7 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 9 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 11 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 13 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 15 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 17 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 19 	Thread : 2 	Event Type : 0  	Variable :  test/Test_y
DataRace at: 
Event Index : 21 	Thread : 2 	Event Type : 0  	